## scikit-learn sample_weight compliance report

This notebook runs compliance tests on all scikit-learn estimators. Estimator as inspected to check whether they are expected to have a stochastic fit or not. If the fit is stochastic, a dedicated statistical test is performed, otherwise a deterministic estimator check is run instead.

In [1]:
import sklearn

sklearn.show_versions()


System:
    python: 3.13.11 | packaged by conda-forge | (main, Dec  6 2025, 11:28:54) [Clang 19.1.7 ]
executable: /Users/ogrisel/miniforge3/envs/dev/bin/python
   machine: macOS-15.6.1-arm64-arm-64bit-Mach-O

Python dependencies:
      sklearn: 1.9.dev0
          pip: 25.3
   setuptools: 80.9.0
        numpy: 2.4.0
        scipy: 1.16.3
       Cython: 3.2.4
       pandas: 3.0.0.dev0+2566.g2bb3fef887
   matplotlib: 3.10.8
       joblib: 1.3.0
threadpoolctl: 3.6.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 10
         prefix: libopenblas
       filepath: /Users/ogrisel/miniforge3/envs/dev/lib/libopenblas.0.dylib
        version: 0.3.30
threading_layer: openmp
   architecture: VORTEX

       user_api: openmp
   internal_api: openmp
    num_threads: 10
         prefix: libomp
       filepath: /Users/ogrisel/miniforge3/envs/dev/lib/libomp.dylib
        version: None


In [2]:
from inspect import signature
import traceback
import warnings
import pandas as pd
from sklearn.utils import all_estimators
from sklearn.exceptions import ConvergenceWarning
import threadpoolctl

from sample_weight_audit import check_weighted_repeated_estimator_fit_equivalence
from sample_weight_audit.sklearn_stochastic_params import STOCHASTIC_FIT_PARAMS

# HistGradientBoostingClassifier trashes the OpenMP thread pool on repeated
# small fits.
threadpoolctl.threadpool_limits(limits=1, user_api="openmp")
warnings.filterwarnings("ignore", category=RuntimeWarning)  # division by zero in AdaBoost
warnings.filterwarnings("ignore", category=ConvergenceWarning)  # liblinear can fail to converge
warnings.filterwarnings("ignore", category=UserWarning)  # KBinsDiscretizer with collapsed bins

In [3]:
from sklearn.linear_model import LogisticRegressionCV

ESTIMATORS_TO_SKIP = [
    LogisticRegressionCV,  # too slow and already somewhat tested by LogisticRegression
]

In [4]:
N_STOCHASTIC_FITS = 1_000
N_SAMPLES_PER_CV_GROUP = 500
N_STATISTICAL_TESTS = 58  # measured by running the script
BONFERRONI_CORRECTION = 1 / N_STATISTICAL_TESTS
TEST_THRESHOLD = 0.05 * BONFERRONI_CORRECTION


statistical_test_results = []
missing_sample_weight_support = []
errors = []


for est_name, est_class in all_estimators(
    type_filter=["classifier", "regressor", "cluster", "transformer"]
):
    if est_class in ESTIMATORS_TO_SKIP:
        print(f"Skipping {est_name}")
        continue

    if "sample_weight" not in signature(est_class.fit).parameters:
        print(f"⚠ {est_name} does not support sample_weight")
        missing_sample_weight_support.append(est_name)
        continue

    try:
        est = est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))
    except TypeError as e:
        print(f"⚠ {est_name} failed to instantiate: {e}")
        continue

    print(f"Evaluating {est}")
    try:
        result = check_weighted_repeated_estimator_fit_equivalence(
            est,
            est_name,
            test_name="kstest",
            n_stochastic_fits=N_STOCHASTIC_FITS,
            n_samples_per_cv_group=N_SAMPLES_PER_CV_GROUP,
            random_state=0,
        )
        pass_or_fail = "✅" if result.pvalue > TEST_THRESHOLD else "❌"
        print(f"{pass_or_fail} {est_name}: (pvalue: {result.pvalue:.3f})")
        statistical_test_results.append(result)
    except Exception as e:
        print(f"❌ {est} error with: {e}")
        errors.append((est, e))

results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

⚠ ARDRegression does not support sample_weight
Evaluating AdaBoostClassifier(estimator=DecisionTreeClassifier(max_features=0.5,
                                                    min_weight_fraction_leaf=0.1))


100%|██████████| 1000/1000 [01:06<00:00, 15.10it/s]


✅ AdaBoostClassifier: (pvalue: 0.685)
Evaluating AdaBoostRegressor(estimator=DecisionTreeRegressor(max_features=0.5,
                                                  min_weight_fraction_leaf=0.1))


100%|██████████| 1000/1000 [00:57<00:00, 17.37it/s]


✅ AdaBoostRegressor: (pvalue: 0.038)
⚠ AdditiveChi2Sampler does not support sample_weight
⚠ AffinityPropagation does not support sample_weight
⚠ AgglomerativeClustering does not support sample_weight
Evaluating BaggingClassifier(estimator=LogisticRegression(), max_samples=1.0)


...

✅ SplineTransformer: (pvalue: 1.000)
⚠ StackingClassifier does not support sample_weight
⚠ StackingRegressor does not support sample_weight
Evaluating StandardScaler()


100%|██████████| 1/1 [00:00<00:00, 230.61it/s]


✅ StandardScaler: (pvalue: 1.000)
⚠ TSNE does not support sample_weight
⚠ TargetEncoder does not support sample_weight
⚠ TfidfTransformer does not support sample_weight
⚠ TheilSenRegressor does not support sample_weight
⚠ TransformedTargetRegressor does not support sample_weight
⚠ TruncatedSVD does not support sample_weight
⚠ TunedThresholdClassifierCV does not support sample_weight
Evaluating TweedieRegressor()


100%|██████████| 1/1 [00:00<00:00, 185.47it/s]

✅ TweedieRegressor: (pvalue: 1.000)
⚠ VarianceThreshold does not support sample_weight
⚠ VotingClassifier does not support sample_weight
⚠ VotingRegressor does not support sample_weight


In [5]:
print(
    f"✅ {len([r for r in statistical_test_results if r.pvalue > TEST_THRESHOLD])} "
    "passed the statistical test"
)
print(
    f"❌ {len([r for r in statistical_test_results if r.pvalue <= TEST_THRESHOLD])} "
    "failed the statistical test"
)
print(f"❌ {len(errors)} other errors")
print(
    f"⚠ {len(missing_sample_weight_support)} estimators lack sample_weight "
    "support"
)
results_df = pd.DataFrame([r.to_dict() for r in statistical_test_results])

✅ 47 passed the statistical test
❌ 11 failed the statistical test
❌ 3 other errors
⚠ 116 estimators lack sample_weight support


## Details on the statistical test results

In [6]:
results_df.sort_values("pvalue")[["estimator_name", "pvalue", "deterministic_predictions"]]

,estimator_name,pvalue,deterministic_predictions
41,NuSVC,0.000000e+00,False
53,SVC,0.000000e+00,False
37,MLPClassifier,7.621911e-140,False
35,LinearSVR,2.918596e-125,False
34,LinearSVC,9.324432e-123,False
36,LogisticRegression,1.558636e-46,False
26,HistGradientBoostingRegressor,3.423675e-29,False
39,MiniBatchKMeans,7.709567e-09,False
49,RidgeClassifier,1.477668e-07,False
25,HistGradientBoostingClassifier,1.909289e-07,False


## Details on errors

In [7]:
import sys

for est, e in errors:
    print(f"❌ {est}: {e}")
    traceback.print_exception(e, file=sys.stdout)
    print()

❌ IsotonicRegression(): 
Traceback (most recent call last):
  File "/var/folders/93/0_k_9bh97fj_15hzk2bhy5r00000gn/T/ipykernel_11597/1843924585.py", line 33, in <module>
    result = check_weighted_repeated_estimator_fit_equivalence(
        est,
    ...<4 lines>...
        random_state=0,
    )
  File "/Users/ogrisel/code/sample-weight-audit-nondet/src/sample_weight_audit/estimator_check.py", line 96, in check_weighted_repeated_estimator_fit_equivalence
    multifit_results = multifit_over_weighted_and_repeated(
        est,
    ...<7 lines>...
        random_state=random_state,
    )
  File "/Users/ogrisel/code/sample-weight-audit-nondet/src/sample_weight_audit/estimator_check.py", line 394, in multifit_over_weighted_and_repeated
    assert predictions_repeated_all.shape[1] == X_test.shape[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError

❌ RANSACRegressor(): Sample weights must contain at least one non-zero number.
Traceback (most recent call last):


## List of estimators with missing sample_weight support

In [8]:
for est_name in missing_sample_weight_support:
    print(est_name)

ARDRegression
AdditiveChi2Sampler
AffinityPropagation
AgglomerativeClustering
BernoulliRBM
Binarizer
Birch
CCA
ClassifierChain
ColumnTransformer
DictVectorizer
DictionaryLearning
FactorAnalysis
FastICA
FeatureAgglomeration
FeatureHasher
FeatureUnion
FixedThresholdClassifier
FunctionTransformer
GaussianProcessClassifier
GaussianProcessRegressor
GaussianRandomProjection
GenericUnivariateSelect
HDBSCAN
HashingVectorizer
IncrementalPCA
Isomap
KNNImputer
KNeighborsClassifier
KNeighborsRegressor
KNeighborsTransformer
KernelCenterer
KernelPCA
LabelBinarizer
LabelEncoder
LabelPropagation
LabelSpreading
Lars
LarsCV
LassoLars
LassoLarsCV
LassoLarsIC
LatentDirichletAllocation
LinearDiscriminantAnalysis
LocallyLinearEmbedding
MaxAbsScaler
MeanShift
MinMaxScaler
MiniBatchDictionaryLearning
MiniBatchNMF
MiniBatchSparsePCA
MissingIndicator
MultiLabelBinarizer
MultiTaskElasticNet
MultiTaskElasticNetCV
MultiTaskLasso
MultiTaskLassoCV
NMF
NearestCentroid
NeighborhoodComponentsAnalysis
Normalizer
Nystroe

## Example interactive inspection of the predictions and scores of a given estimator

In [9]:
results = next(r for r in statistical_test_results if r.estimator_name == "GaussianNB")
results.predictions_repeated.shape

(1, 1000, 3)

In [10]:
results.predictions_weighted.shape

(1, 1000, 3)

In [11]:
import numpy as np
np.abs(results.predictions_repeated - results.predictions_weighted).max()

np.float64(1.000000082740371e-10)

In [12]:
np.abs(results.scores_repeated - results.scores_weighted).max()

np.float64(1.4867551634267784e-11)

In [13]:
from scipy.stats import kstest
kstest(results.scores_repeated, results.scores_weighted)

KstestResult(statistic=np.float64(1.0), pvalue=np.float64(1.0), statistic_location=np.float64(0.9978075242446157), statistic_sign=np.int8(1))